<a href="https://colab.research.google.com/github/lisatwyw/SMILES-X/blob/main/AlpacaLlama31.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# prompt: mount google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%%time
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" --quiet
!pip install --no-deps xformers trl peft accelerate bitsandbytes --quiet

In [3]:
%%time

import torch
from unsloth import FastLanguageModel
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

MODEL_ID = 'llama31'
# MODEL_ID = 'mistral'

if MODEL_ID == 'llama31':
    # 4bit pre quantized models we support for 4x faster downloading + no OOMs.
    fourbit_models = [
        "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
        "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
        "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
        "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
        "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
        "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
        "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
        "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
        "unsloth/Phi-3-mini-4k-instruct",          # Phi-3 2x faster!d
        "unsloth/Phi-3-medium-4k-instruct",
        "unsloth/gemma-2-9b-bnb-4bit",
        "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
    ] # More models at https://huggingface.co/unsloth

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "unsloth/Meta-Llama-3.1-8B",
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
        # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
    )
else:

    # 4bit pre quantized models we support for 4x faster downloading + no OOMs.
    fourbit_models = [
        "unsloth/mistral-7b-bnb-4bit",
        "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
        "unsloth/llama-2-7b-bnb-4bit",
        "unsloth/llama-2-13b-bnb-4bit",
        "unsloth/codellama-34b-bnb-4bit",
        "unsloth/tinyllama-bnb-4bit",
    ] # More models at https://huggingface.co/unsloth

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "unsloth/mistral-7b-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
        # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
    )

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.7: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers, TRL and unsloth via:
`pip install --upgrade --no-cache-dir unsloth git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/trl.git`


CPU times: user 35.7 s, sys: 24.2 s, total: 60 s
Wall time: 1min 49s


In [6]:
%%time

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Already have LoRA adapters! We shall skip this step.


CPU times: user 3.43 ms, sys: 1.22 ms, total: 4.65 ms
Wall time: 4.32 ms


In [5]:
 %%time

 alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

alpaca_data_cleaned.json:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

CPU times: user 1.02 s, sys: 381 ms, total: 1.4 s
Wall time: 5.89 s


In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 101, # Set num_train_epochs = 1 for full training runs
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/51760 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 101
 "-____-"     Number of trainable parameters = 41,943,040


**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers, TRL and Unsloth!
`pip install --upgrade --no-cache-dir unsloth git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/trl.git`


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,1.817600
2,2.304200
3,1.689200
4,1.938400
5,1.657000
6,1.621800
7,1.187300
8,1.263500
9,1.100000
10,1.187700


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [9]:
model.save_pretrained( MODEL_ID ) # Local saving
tokenizer.save_pretrained( MODEL_ID )

('llama31/tokenizer_config.json',
 'llama31/special_tokens_map.json',
 'llama31/tokenizer.json')

In [10]:
instruction = '''
Using literature and databases of authors of autobiographies, what is the ethnicity of the following names "Firstname Lastname"? Be concise.

The ethnicity class labels are:
0 - Aboriginal: includes First Nations, Inuit (Inuk), Métis, North American Indian.
1 - East Asian; includes Chinese, Taiwanese, Japanese, Korean
2 - South Asian: includes Bangladeshi, Indian, Pakistan, Nepalese, Sri Lankan
3 - Southeast Asian: includes Cambodian, Filipino, Laotian, Malaysian, Thai, Vietnamese.
4 - Latin American: includes Hispanic.
5 - Middle Eastern: includes Afghan, Arab, Iranian, Muslim, West Asian.
6 - Not a visible minority: includes White.
7 - Mixed
'''
print( instruction )



Using literature and databases of authors of autobiographies, what is the ethnicity of the following names "Firstname Lastname"? Be concise.

The ethnicity class labels are: 
0 - Aboriginal: includes First Nations, Inuit (Inuk), Métis, North American Indian.
1 - East Asian; includes Chinese, Taiwanese, Japanese, Korean
2 - South Asian: includes Bangladeshi, Indian, Pakistan, Nepalese, Sri Lankan
3 - Southeast Asian: includes Cambodian, Filipino, Laotian, Malaysian, Thai, Vietnamese.
4 - Latin American: includes Hispanic.
5 - Middle Eastern: includes Afghan, Arab, Iranian, Muslim, West Asian.
6 - Not a visible minority: includes White.
7 - Mixed



In [11]:
for name in ['Blll Gates', 'Donald Trump']:
    # alpaca_prompt = Copied from above
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            instruction, # instruction
            name, # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    from transformers import TextStreamer
    text_streamer = TextStreamer(tokenizer, skip_prompt = True )
    answer = model.generate(input_ids = inputs.input_ids, attention_mask = inputs.attention_mask,
                       streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

The ethnicity of Bill Gates is White.<|end_of_text|>
Donald Trump is a Caucasian American, and therefore his ethnicity is classified as "Not a visible minority" (label 6).<|end_of_text|>


In [19]:
instruction = """
You are an Onomastic system that analyzes names and databases of autobiographies to map a person's name to the person's ethinicity class.
The system should reply as '### ANSWER: CLASS[X]'.

The ethnicity class labels are:
CLASS0 - Aboriginal: includes First Nations, Inuit (Inuk), Métis, North American Indian.
CLASS1 - East Asian; includes Chinese, Taiwanese, Japanese, Korean
CLASS2 - South Asian: includes Bangladeshi, Indian, Pakistan, Nepalese, Sri Lankan
CLASS3 - Southeast Asian: includes Cambodian, Filipino, Laotian, Malaysian, Thai, Vietnamese.
CLASS4 - Latin American: includes Hispanic.
CLASS5 - Middle Eastern: includes Afghan, Arab, Iranian, Muslim, West Asian.
CLASS6 - Not a visible minority: includes White.
CLASS7 - Mixed
"""

print( instruction)


You are an Onomastic system that analyzes names and databases of autobiographies to map a person's name to the person's ethinicity class. 
The system should reply as '### ANSWER: CLASS[X]'. 

The ethnicity class labels are: 
CLASS0 - Aboriginal: includes First Nations, Inuit (Inuk), Métis, North American Indian.
CLASS1 - East Asian; includes Chinese, Taiwanese, Japanese, Korean
CLASS2 - South Asian: includes Bangladeshi, Indian, Pakistan, Nepalese, Sri Lankan
CLASS3 - Southeast Asian: includes Cambodian, Filipino, Laotian, Malaysian, Thai, Vietnamese.
CLASS4 - Latin American: includes Hispanic.
CLASS5 - Middle Eastern: includes Afghan, Arab, Iranian, Muslim, West Asian.
CLASS6 - Not a visible minority: includes White.
CLASS7 - Mixed



In [20]:
for name in ['Kei McDonald', 'Jared Williams']:
    # alpaca_prompt = Copied from above
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            instruction, # instruction
            name, # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    from transformers import TextStreamer
    text_streamer = TextStreamer(tokenizer, skip_prompt = True )
    answer = model.generate(input_ids = inputs.input_ids, attention_mask = inputs.attention_mask,
                       streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

### ANSWER: CLASS6<|end_of_text|>
### ANSWER: CLASS6.<|end_of_text|>


In [22]:
for name in ['Melody Charlie', 'Jay Peters', 'Lisa Tang', 'Naveed Janjua', 'Hind Sbihi', 'Michelle Ki', 'Sara Sarabosing']:
    # alpaca_prompt = Copied from above
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            instruction, # instruction
            name, # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    from transformers import TextStreamer
    text_streamer = TextStreamer(tokenizer, skip_prompt = True )
    answer = model.generate(input_ids = inputs.input_ids, attention_mask = inputs.attention_mask,
                       streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

### ANSWER: CLASS6<|end_of_text|>
### ANSWER: CLASS6<|end_of_text|>
### ANSWER: CLASS7<|end_of_text|>
### ANSWER: CLASS5<|end_of_text|>
### ANSWER: CLASS2<|end_of_text|>
### ANSWER: CLASS0<|end_of_text|>
### ANSWER: CLASS7<|end_of_text|>
